# Imports 

In [106]:
import pandas as pd
import numpy as np
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Reshaping 

In [107]:
## simulate data
np.random.seed(1129)
shop_names = ['Compass', 'Starbucks', 'Baked and Wired', 'Peets', 
              'Blue Bottle', 'Saxbys']
coffee_df = pd.concat([pd.DataFrame({'shop_name': shop_names,
                         'opening_time': np.random.choice(["8:00 AM", "9:00 AM", "10:00 AM"],
                                                       len(shop_names),
                                                       replace = True),
                         'closing_time': np.random.choice(["5:00 PM", "6:00 PM", 
                                                          "7:00 PM"],
                                                         len(shop_names),
                                                          replace = True),
                         'hourly_wage': np.random.uniform(14, 20,
                                                          len(shop_names)),
                        'year': 2019}),
                      pd.DataFrame({'shop_name': shop_names,
                         'opening_time': np.random.choice(["8:00 AM", "9:00 AM", "10:00 AM"],
                                                       len(shop_names),
                                                       replace = True),
                         'closing_time': np.random.choice(["3:00 PM", "4:00 PM",
                                                           "6:00 PM", 
                                                          "7:00 PM"],
                                                         len(shop_names),
                                                          replace = True),
                         'hourly_wage': np.random.uniform(14, 20,
                                                          len(shop_names)),
                        'year': 2021})]).sort_values(by = 'shop_name')
                      

coffee_df

,shop_name,opening_time,closing_time,hourly_wage,year
2,Baked and Wired,10:00 AM,5:00 PM,16.308214,2019
2,Baked and Wired,10:00 AM,3:00 PM,19.222221,2021
4,Blue Bottle,8:00 AM,7:00 PM,19.231655,2019
4,Blue Bottle,9:00 AM,7:00 PM,19.951942,2021
0,Compass,10:00 AM,6:00 PM,16.169551,2019
0,Compass,10:00 AM,7:00 PM,15.316723,2021
3,Peets,10:00 AM,7:00 PM,16.240656,2019
3,Peets,8:00 AM,7:00 PM,19.992475,2021
5,Saxbys,9:00 AM,6:00 PM,14.598841,2019
5,Saxbys,8:00 AM,3:00 PM,17.238755,2021


## 1.1 Create a wide-format dataframe where we have the hourly wage for each year as a separate column

In [108]:
coffee_df_wide = pd.pivot(coffee_df, index='shop_name', columns='year', values='hourly_wage').reset_index()
coffee_df_wide.columns = ['shop_name', 'wage_19', 'wage_21']
coffee_df_wide

,shop_name,wage_19,wage_21
0,Baked and Wired,16.308214,19.222221
1,Blue Bottle,19.231655,19.951942
2,Compass,16.169551,15.316723
3,Peets,16.240656,19.992475
4,Saxbys,14.598841,17.238755
5,Starbucks,19.470102,19.728897


## 1.2 Create a long-format dataframe where we have the hourly wage for each year as a separate column

In [109]:
coffee_df_long = pd.melt(coffee_df_wide, id_vars = 'shop_name').sort_values(by = 'shop_name')
coffee_df_long['variable'] = coffee_df_long['variable'].str.replace('wage_', '20')
coffee_df_long.columns = ['shop_name', 'year', 'hourly_wage']
coffee_df_long

,shop_name,year,hourly_wage
0,Baked and Wired,2019,16.308214
6,Baked and Wired,2021,19.222221
1,Blue Bottle,2019,19.231655
7,Blue Bottle,2021,19.951942
2,Compass,2019,16.169551
8,Compass,2021,15.316723
3,Peets,2019,16.240656
9,Peets,2021,19.992475
4,Saxbys,2019,14.598841
10,Saxbys,2021,17.238755


## 1.3 Practice for you

Using the original `coffee_df`:

- Reshape to wide format for opening_time and closing_time (you can remove hourly_wage) from the data
- Use list comprehension to subset the dataframe to the shop_name and 2019 times only (so shop_name, opening_time 2019 and closing time 2019)

In [110]:
coffee_df_wide_hours = pd.pivot(coffee_df, index='shop_name', columns='year', values=['opening_time', 'closing_time']).reset_index()
coffee_df_wide_hours.columns = [str(x) if str(x) == "shop_name" else (str(x) + "_" + str(y)) for x, y in coffee_df_wide_hours.columns]
coffee_df_wide_hours_2019 = coffee_df_wide_hours[['shop_name', 'opening_time_2019', 'closing_time_2019']]
coffee_df_wide_hours_2019

,shop_name,opening_time_2019,closing_time_2019
0,Baked and Wired,10:00 AM,5:00 PM
1,Blue Bottle,8:00 AM,7:00 PM
2,Compass,10:00 AM,6:00 PM
3,Peets,10:00 AM,7:00 PM
4,Saxbys,9:00 AM,6:00 PM
5,Starbucks,9:00 AM,7:00 PM


# 2. Merging


## 2.1: Read in the two datasets 

- San Diego data: `naics_code` and `account_key`
- NAICS details data: `naics` 

Run code below; if pulling from github, pathname should be fine; if working elsewhere may need to edit path name at read in 

In [111]:
sd_df = pd.read_csv("../public_data/sd_df.csv")
naics_df = pd.read_csv("../public_data/naics_df.csv")

In [112]:
cols_sd_use = ["naics_code", "account_key"]
cols_naics_use = ["naics"]

# make sure relevant columns are string/character 
sd_df[cols_sd_use] = sd_df[cols_sd_use].astype(str)
naics_df[cols_naics_use] = naics_df[cols_naics_use].astype(str)

sd_df.head()
naics_df.head()

,account_key,dba_name,council_district,naics_code,naics_description,naics_nchar
0,1974000448,ERNST & YOUNG LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
1,1974011093,HECHT SOLBERG ROBINSON GOLDBERG & BAGLEY LLP,cd_3,5411,LEGAL SERVICES,4
2,1978039819,RSM US LLP,cd_1,541211,OFFICES OF CERTIFIED PUBLIC ACCOUNTANTS,6
3,1978042092,THORSNES BARTOLOTTA MCGUIRE LLP,cd_3,5411,LEGAL SERVICES,4
4,1979046817,KORENIC & WOJDOWSKI LLP,cd_7,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4


,naics,naics_description
0,111140,Wheat Farming
1,111160,Rice Farming
2,111150,Corn Farming
3,111110,Soybean Farming
4,111120,Oilseed (except Soybean) Farming


## 2.2 "Inner join"- retain only San Diego businesses with details on their NAICS code

- Use the `naics_code` column in the San Diego business data as the join key
- Use the `naics` column in the NAICS code details data as the join key

- Do an inner join of the San Diego data onto the NAICS code details using these join keys
- After the inner join, print some examples of San Diego businesses lost in the merge
- Use value_counts() on the `naics_nchar` column in the San Diego data to see why they might have gotten lost


In [113]:
sd_naics_inner = pd.merge(sd_df, naics_df, left_on = "naics_code", right_on = "naics", how = "inner")

f"in our original data, we had {sd_df.shape[0]} rows, but now have {sd_naics_inner.shape[0]} rows after merging with naics_df"

lost_naics = sd_df.loc[~sd_df["account_key"].isin(sd_naics_inner["account_key"]),
                     ['naics_code', 'naics_description', 'naics_nchar']].drop_duplicates()
lost_naics.head()
lost_naics["naics_nchar"].value_counts()

'in our original data, we had 353 rows, but now have 52 rows after merging with naics_df'

,naics_code,naics_description,naics_nchar
1,5411,LEGAL SERVICES,4
4,5412,ACCOUNTING/TAX PREP/BOOKKEEP/PAYROLL SERVICES,4
12,7221,FULL-SERVICE RESTAURANTS,4
13,48841,MOTOR VEHICLE TOWING,5
14,54111,OFFICES OF LAWYERS,5


5    56
4    26
3    16
6     9
2     7
Name: naics_nchar, dtype: int64

## 2.3 "Left join"- retain all sd businesses even if naics code isn't in the naics_details df

- Using the same join keys as above, and treating the San Diego businesses as the left hand side data, left join the naics code details onto the San Diego businesses
- Use the `indicator` argument within merge to create an indicator, `naics_merge_status`, to help with later merge diagnostics and examine sample of ones that didn't merge]
- Use the `suffixes` argument within merge to add `_sd` as the left suffix, `_census` as the right suffix


In [114]:
sd_naics_left = pd.merge(sd_df, naics_df, left_on = "naics_code", right_on = "naics", how = "left", suffixes=("_sd", "_census"), indicator = "merged_with_naics")
sd_naics_left["merged_with_naics"].value_counts(normalize=True)

'A sample of five rows that did not merge with naics_df:'
sd_naics_left[sd_naics_left["merged_with_naics"] == "left_only"].sample(5)

left_only     0.859459
both          0.140541
right_only    0.000000
Name: merged_with_naics, dtype: float64

'A sample of five rows that did not merge with naics_df:'

,account_key,dba_name,council_district,naics_code,naics_description_sd,naics_nchar,naics,naics_description_census,merged_with_naics
92,2006002639,PARKER & CROSLAND LLP,cd_3,54111,OFFICES OF LAWYERS,5,NaN,NaN,left_only
362,2006015404,9625 TOWN CENTRE PARTNERS LP,cd_3,541,"PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES",3,NaN,NaN,left_only
67,2004005238,SCRIPPS MERCY SURGERY PAVILION,cd_3,62111,OFFICES OF PHYSICIANS,5,NaN,NaN,left_only
252,2019004000,HDP WEST PARK LP,cd_3,53111,LESSORS OF RESIDENTIAL BUILDINGS & DWELLINGS,5,NaN,NaN,left_only
319,2020009114,KCG VENTURES LLC,cd_2,454,NONSTORE RETAILERS,3,NaN,NaN,left_only


## 2.4 Use group by and agg to see if there are differences in merge rates by area

- Using the left-joined dataframe created in previous step, create a boolean indicator---`is_lost` if the merge indicator is equal to "left_only"
- Group by `council_district` and use the shortcut of taking the mean of a True/False indicator to find the proportions in order to find the proportion lost in the merge (so in the left join, ones that failed to match to `naics_df`) by council_district


In [115]:
sd_naics_left["is_lost"] = sd_naics_left["merged_with_naics"] == "left_only"
sd_naics_left["is_lost"].value_counts()

sd_naics_left.groupby("council_district").agg({"is_lost": "mean", "account_key": "nunique"}).rename(columns = {"is_lost": "pct_not_lost", "account_key": "num_account_key"}).reset_index()

True     318
False     52
Name: is_lost, dtype: int64

,council_district,pct_not_lost,num_account_key
0,cd_1,0.880597,67
1,cd_2,0.804348,42
2,cd_3,0.812500,103
3,cd_4,1.000000,4
4,cd_5,0.916667,24
5,cd_6,0.859649,54
6,cd_7,0.918919,36
7,cd_8,0.933333,15
8,cd_9,1.000000,8


## 2.5 Practice for you: add lagging 0's and see if merge rate from left join improves

You noticed earlier that a big reason for non-matches is that the San Diego tax certificate NAICS codes were oftentimes not 6-digits long, while the Census ones were always 6 digits

You wonder if this is an issue where 0's in some of the SD's data naics codes got cutoff (eg 540000 became 54) and if adding these lagging zeros would improve the merge rate in a left join

- Using one of two approaches, pad the `naics_code` column in `sd_df` with 0's to get that column up to 6-digits: (1) str.pad in pandas (https://pandas.pydata.org/docs/reference/api/pandas.Series.str.pad.html); (2) for more of a challenge, write your own function that checks the # of digits in the naics code string and pads with 0's at the end up to 6 digits and use row-wise apply---`df.apply(funcname, axis = 1)`---to execute it
- Perform the same left join as in 2.3 and see how the match rate changes
- Create an indicator variable--`is_new_match`---for new matches under the padded NAICS code; compare the `naics_description` column from San Diego versus Census in the new dataset for a sample of these new matches and comment on whether the padding seems to be correct

In [116]:
sd_df["naics_padded"] = sd_df["naics_code"].str.ljust(6, "0")
# From solution: sd_df.naics_code.str.pad(width = 6, side = "right", fillchar = "0")
sd_df[['naics_code', 'naics_padded']].head()

sd_naics_left_padded = pd.merge(sd_df, naics_df, left_on = "naics_padded", right_on = "naics", how = "left", suffixes=("_sd", "_census"), indicator = "merged_with_naics")
sd_naics_left_padded["merged_with_naics"].value_counts(normalize=True)

lost_account_keys = sd_naics_left.loc[sd_naics_left["merged_with_naics"] == "left_only", "account_key"].unique()
"Counts of if a new matches:"
sd_naics_left_padded["is_new_match"] = np.where((sd_naics_left_padded["account_key"].isin(lost_account_keys)) & (sd_naics_left_padded["merged_with_naics"] == "both"), True, False)
sd_naics_left_padded["is_new_match"].value_counts()

sd_naics_left_padded[sd_naics_left_padded["is_new_match"] == True][["dba_name",
               "naics_description_sd", "naics_description_census"]].sample(5)

"By adding the padding, we were able to match 185 more accounts to the census data, so the padding seems to be correct."

,naics_code,naics_padded
0,541211,541211
1,5411,541100
2,541211,541211
3,5411,541100
4,5412,541200


both          0.53139
left_only     0.46861
right_only    0.00000
Name: merged_with_naics, dtype: float64

'Counts of if a new matches:'

False    261
True     185
Name: is_new_match, dtype: int64

,dba_name,naics_description_sd,naics_description_census
166,AGUIRRE & SEVERSON LLP,OFFICES OF LAWYERS,Offices of Lawyers
419,THRIVE AFFORDABLE VET CARE,VETERINARY SERVICES,Veterinary Services
136,STRAZZERI MANCINI LLP,OFFICES OF LAWYERS,Offices of Lawyers
350,WACHIC,OTHER CLOTHING STORES,Other Clothing Stores
375,ODAI AUTO SALES,USED CAR DEALERS,Used Car Dealers


'By adding the padding, we were able to match 185 more accounts to the census data, so the padding seems to be correct.'